In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys

service = Service()
driver = webdriver.Chrome(service=service)

driver.get("https://www.timeforstorm.eu/stormstudio/login/content%20guru/8a46c4ac0c101da5#/login/content%20guru/8a46c4ac0c101da5%20storm%20STUDIO%E2%84%A2")

In [35]:
org_search = driver.find_element(By.CSS_SELECTOR, '#prefix_search input')
org_search.send_keys('Allianz Partners EU')
org_search.send_keys(Keys.ENTER)


In [36]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

user_dropdown = driver.find_element(By.CLASS_NAME, 'el-select')
user_dropdown.click()

admin_user = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//li[contains(@class, 'el-select-dropdown__item') and contains(., 'red01')]"))
)
admin_user.click()

login = driver.find_element(By.ID, "loginButton")
login.click()